## Cargar Datos

Inicialmente esta parte el código importa varias bibliotecas esenciales para el procesamiento de datos y modelado en Python. Pandas se utiliza para trabajar con datos tabulares, NumPy proporciona estructuras de datos para cálculos numéricos eficientes, y scikit-learn ofrece herramientas para el preprocesamiento de datos, como la imputación de valores faltantes y la codificación de variables categóricas. El módulo os facilita la interacción con el sistema operativo, mientras que re permite trabajar con expresiones regulares para manipular cadenas de texto. Estas bibliotecas son fundamentales para la preparación de datos y la construcción de modelos de aprendizaje automático en Python.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import os
import re

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = "."

In [ ]:
!kaggle competitions download -c udea-ai4eng-20241


 61% 13.0M/21.2M [00:00<00:00, 65.9MB/s]
100% 21.2M/21.2M [00:00<00:00, 73.7MB/s]


In [ ]:
!unzip udea-ai4eng-20241.zip

Archive:  udea-ai4eng-20241.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
df = pd.read_csv("train.csv")

In [ ]:
df

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,25096,20195,BIOLOGIA,LA GUAJIRA,Entre 500 mil y menos de 1 millón,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,Si,Si,Si,Secundaria (Bachillerato) incompleta,medio-alto
692496,754213,20212,PSICOLOGIA,NORTE SANTANDER,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Primaria incompleta,Si,No,N,No,Si,Si,Secundaria (Bachillerato) incompleta,bajo
692497,504185,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Menos de 10 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,Si,Si,Si,Secundaria (Bachillerato) incompleta,medio-bajo
692498,986620,20195,PSICOLOGIA,TOLIMA,Entre 2.5 millones y menos de 4 millones,Menos de 10 horas,Estrato 1,No,Primaria completa,No,No,N,Si,Si,No,Primaria completa,bajo


# Pre-Procesamiento de la Información:

En esta parte del preprocesamiento de datos se prepara el conjunto de datos para el análisis y modelado predictivo, mediante varias transformaciones y limpieza:

**Limpieza de Datos (clean_data):**

* Eliminar Columnas Innecesarias:
 Se eliminan columnas irrelevantes ('FAMI_TIENEINTERNET.1', 'ESTU_PRIVADO_LIBERTAD', 'PERIODO') para simplificar el conjunto de datos.
* Se eliminan los prefijos de texto en la columna 'FAMI_ESTRATOVIVIENDA' y se convierten los valores a numéricos, reemplazando "Sin" por 0.

**Conversión de Valores Categóricos a Booleanos:**

Para las columnas indicadas ('FAMI_TIENEINTERNET', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR'), los valores 'Si' y 'No' se convierten en 1 y 0 respectivamente. Además, se llenan valores faltantes con la moda de cada columna.


**Agrupación y Normalización de Programas Académicos:**

Los nombres de los programas académicos se agrupan en categorías más generales (por ejemplo, 'Ingeniería', 'Licenciatura', 'Administración', 'Salud', 'Naturales', 'Artes', 'Finanzas', 'Sociales', 'Letras', 'Construcción', 'Deportes', 'Documentación', 'Hotelerías', 'Publicidad', 'Comunicación').
Esto se logra mediante el uso de expresiones regulares para identificar patrones en los nombres de los programas académicos y reemplazarlos con las categorías correspondientes.

**Codificación One-Hot para Programas Académicos:**

Finalmente, se realiza una codificación one-hot para los programas académicos normalizados y se agregan al conjunto de datos.

También se realizó una **Conversión de Valores Categóricos a Booleanos** e **Imputación de Valores Faltantes** utilizando la moda como estrategia.


In [ ]:
def contiene_ingenieria(word):
    word = word.lower()
    return 'ingenier' in word or 'ing.' in word

def clean_data(data):
  # Dropear Columnas Innecesarias
  data = data.drop(['FAMI_TIENEINTERNET.1'], axis=1)
  data = data.drop('ESTU_PRIVADO_LIBERTAD', axis=1)
  data = data.drop('PERIODO', axis=1)

  # Convertir Si y No a Booleanos
  columnas_booleanas = ['FAMI_TIENEINTERNET', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL',
                      'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR']

  for columna in columnas_booleanas:
    moda = data[columna].mode()[0]
    data[columna].fillna(moda, inplace=True)
    data[columna] = data[columna].map({'Si': 1, 'No': 0}).astype(int)

  # Borrar Strings de ESTRATO VIVIENDA
  data['FAMI_ESTRATOVIVIENDA'] = data['FAMI_ESTRATOVIVIENDA'].str.replace('Estrato', '').str.strip()
  data['FAMI_ESTRATOVIVIENDA'].replace('Sin', '0', inplace=True)

  # Convertir la columna a tipo numérico
  data['FAMI_ESTRATOVIVIENDA'] = pd.to_numeric(data['FAMI_ESTRATOVIVIENDA'])

  # One Hot Coding a VALOR MATRICULA
  encoded_df = pd.get_dummies(data['ESTU_VALORMATRICULAUNIVERSIDAD']).astype(int)
  data = pd.concat([data, encoded_df], axis=1)
  data.drop('ESTU_VALORMATRICULAUNIVERSIDAD', axis=1, inplace=True)

  #Rangos de ESTU_HORASSEMANA
  encoded_df = pd.get_dummies(data['ESTU_HORASSEMANATRABAJA'].astype(str), prefix='Horas_Trabajo').astype(int)
  data = pd.concat([data, encoded_df], axis=1)
  data.drop('ESTU_HORASSEMANATRABAJA', axis=1, inplace=True)

  #Imputar Datos Faltantes:
  moda_estrato = data['FAMI_ESTRATOVIVIENDA'].mode()[0]  # Obtener la moda
  data['FAMI_ESTRATOVIVIENDA'].fillna(moda_estrato, inplace=True)
  data['FAMI_ESTRATOVIVIENDA'] = data['FAMI_ESTRATOVIVIENDA'].astype(int)

  #Rangos de Madre y Padre
  moda_padre = data['FAMI_EDUCACIONPADRE'].mode()[0]
  data['FAMI_EDUCACIONPADRE'].fillna(moda_padre, inplace=True)

  moda_madre = data['FAMI_EDUCACIONMADRE'].mode()[0]
  data['FAMI_EDUCACIONMADRE'].fillna(moda_madre, inplace=True)

  # Eliminar las columnas originales de educación padre y madre
  data.drop(['FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE'], axis=1, inplace=True)
  data.drop(['FAMI_TIENELAVADORA'], axis=1, inplace=True)
  data.drop(['FAMI_TIENEAUTOMOVIL'], axis=1, inplace=True)

  moda_programa = data['ESTU_PRGM_DEPARTAMENTO'].mode()[0]
  data['ESTU_PRGM_DEPARTAMENTO'].fillna(moda_programa, inplace=True)

  #hot para los departamentos
  encoded_df = pd.get_dummies(data['ESTU_PRGM_DEPARTAMENTO'].astype(str), prefix='DEPARTAMENTO_').astype(int)
  data = pd.concat([data, encoded_df], axis=1)
  data.drop('ESTU_PRGM_DEPARTAMENTO', axis=1, inplace=True)

  #Hot Estrato
  encoded_df = pd.get_dummies(data['FAMI_ESTRATOVIVIENDA'].astype(str), prefix='ESTRATO_').astype(int)
  data = pd.concat([data, encoded_df], axis=1)
  data.drop('FAMI_ESTRATOVIVIENDA', axis=1, inplace=True)

  #modificamos las palabras ingenieria
  mask = data['ESTU_PRGM_ACADEMICO'].apply(contiene_ingenieria)

  #agrupamos todas las carreras
  data.loc[mask, 'ESTU_PRGM_ACADEMICO'] = 'Ingeniería'

  pattern = re.compile(r'^LICENCIATURA', flags=re.IGNORECASE)
  mask = data['ESTU_PRGM_ACADEMICO'].str.contains(pattern)
  data.loc[mask, 'ESTU_PRGM_ACADEMICO'] = 'Licenciatura'

  pattern = re.compile(r'^ADMINISTRACI\s*N', flags=re.IGNORECASE)
  mask = data['ESTU_PRGM_ACADEMICO'].str.contains(pattern)
  data.loc[mask, 'ESTU_PRGM_ACADEMICO'] = 'Administración'

  pattern = re.compile(r'^ADMINISTRACI\s*N', flags=re.IGNORECASE)
  mask = data['ESTU_PRGM_ACADEMICO'].str.contains(pattern)
  data.loc[mask, 'ESTU_PRGM_ACADEMICO'] = 'Administración'

  pattern = re.compile(r'^COMUNICACI\s*N', flags=re.IGNORECASE)
  mask = data['ESTU_PRGM_ACADEMICO'].str.contains(pattern)
  data.loc[mask, 'ESTU_PRGM_ACADEMICO'] = 'Comunicación'

  pattern = re.compile(r'^CIENCIAS', flags=re.IGNORECASE)
  mask = data['ESTU_PRGM_ACADEMICO'].str.contains(pattern)
  data.loc[mask, 'ESTU_PRGM_ACADEMICO'] = 'Ciencias'

  pattern = re.compile(r'DISE\s*O', flags=re.IGNORECASE)
  mask = data['ESTU_PRGM_ACADEMICO'].str.contains(pattern)
  data.loc[mask, 'ESTU_PRGM_ACADEMICO'] = 'Diseño'

  pattern = re.compile(r'NEGOCIOS', flags=re.IGNORECASE)
  mask = data['ESTU_PRGM_ACADEMICO'].str.contains(pattern)
  data.loc[mask, 'ESTU_PRGM_ACADEMICO'] = 'Negocios'

  pattern = re.compile(r'PROFESIONAL', flags=re.IGNORECASE)
  mask = data['ESTU_PRGM_ACADEMICO'].str.contains(pattern)
  data.loc[mask, 'ESTU_PRGM_ACADEMICO'] = 'Profesional'

  pattern = re.compile(r'ESTUDIOS', flags=re.IGNORECASE)
  mask = data['ESTU_PRGM_ACADEMICO'].str.contains(pattern)
  data.loc[mask, 'ESTU_PRGM_ACADEMICO'] = 'Estudios'

  carreras = [
    'enfermeria', 'medicina veterinaria', 'seguridad y salud en el trabajo', 'medicina',
    'odontologia', 'medicina veterinaria y zootecnia', 'fisioterapia', 'farmacia',
    'nutricion y dietetica', 'terapia respiratoria', 'enfermer a', 'terapia ocupacional',
    'intrumentacion quirurgica', 'bacteriologia', 'bacteriologia y laboratorio clinico',
    'fonoaudiologia', 'microbiologia y bioanalisis', 'salud ocupacional', 'nutrici n y diet tica',
    'microbiologia', 'microbiologia industrial y ambiental', 'odontolog a', 'terapia cardiorrespiratoria',
    'microbiolog a', 'biotecnologia', 'nutrici n y dietetica', 'microbiolog a industrial y ambiental',
    'microbiolog a y bioan lisis', 'quimica farmaceutica', 'qu mica farmac utica', 'optometr a',
    'instrumentacion quirurgica', 'gerontologia', 'optometria', 'gerencia en sistemas de informacion en salud']

  for carrera in carreras:
      pattern = re.compile(carrera, flags=re.IGNORECASE)
      mask = data['ESTU_PRGM_ACADEMICO'].str.contains(pattern)
      data.loc[mask, 'ESTU_PRGM_ACADEMICO'] = 'Salud'

  carreras = [
    'zootecnia', 'biologia', 'geologia', 'quimica', 'ecologia de zonas costeras', 'arqueologia',
    'biolog a', 'geografia aplicada a la organizacion del espacio y planificacion regional', 'fisica',
    'quimica industrial', 'produccion agroindustrial', 'geografia', 'biologia aplicada', 'ecologia',
    'geociencias', 'geolog a', 'f sica', 'agronomia del tropico humedo', 'biologia marina',
    'oceanografia fisica', 'biolog a ambiental', 'qu mica industrial', 'acuicultura', 'astronomia',
    'bioquimica', 'quimica ambiental', 'ecolog a', 'biologia ambiental', 'oceanograf a', 'qu mica', 'agronomia',
    'computaci n cient fica', 'licenc en educacion basica primaria enf en ciencias naturaleseduc ambiental matema lengua castellana', 'agroindustria']

  for carrera in carreras:
      pattern = re.compile(carrera, flags=re.IGNORECASE)
      mask = data['ESTU_PRGM_ACADEMICO'].str.contains(pattern)
      data.loc[mask, 'ESTU_PRGM_ACADEMICO'] = 'Naturales'

  carreras = [
    'maestro en m sica', 'arte danzario', 'artes plasticas', 'artes pl sticas y visuales', 'cine',
    'artes esc nicas', 'musica', 'cine y comunicaci n digital', 'cine y television',
    'direccion y produccion de cine y television', 'artes musicales', 'fotograf a y comunicaci n visual',
    'arte dramatico', 'direccion y produccion de radio y television', 'musica instrumental',
    'artes escenicas', 'm sica', 'artes audiovisuales', 'arte', 'cine y televisi n', 'cine y audiovisuales',
    'artes plasticas y visuales', 'realizaci n en animaci n', 'musica instrumento', 'maestro en musica',
    'creacion literaria', 'musica canto', 'fotograf a', 'artes plasticas con enfasis en procesos bidimensionales y tridimensionales',
    'historia del arte', 'narrativas digitales', 'direccion de banda', 'teatro musical', 'artes pl sticas',
    'animaci n', 'bellas artes', 'diseno grafico', 'direccion y produccion de medios audiovisuales', 'danza']

  for carrera in carreras:
      pattern = re.compile(carrera, flags=re.IGNORECASE)
      mask = data['ESTU_PRGM_ACADEMICO'].str.contains(pattern)
      data.loc[mask, 'ESTU_PRGM_ACADEMICO'] = 'Artes'

  carreras_finanzas = [
      'contaduria p blica', 'estadistica', 'contaduria publica', 'contadur a p blica',
      'finanzas y comercio internacional', 'finanzas y comercio exterior', 'comercio exterior',
      'comercio internacional y finanzas', 'finanzas y relaciones internacionales', 'finanzas',
      'contaduria publica y finanzas internacionales', 'mercadeo publicidad y ventas', 'contaduria',
      'mercadologia', 'contadur a publica', 'matematicas con enfasis en estadistica', 'estad stica',
      'matem ticas', 'contaduria publica enfasis en sistemas y economia solidaria',
      'matem ticas aplicadas y ciencias de la computaci n', 'mercadeo y ventas',
      'matematicas aplicadas', 'comercio y finanzas internacionales', 'banca y finanzas internacionales',
      'matem tica aplicada', 'contadur a internacional', 'matematicas']

  for carrera in carreras_finanzas:
      pattern = re.compile(carrera, flags=re.IGNORECASE)
      mask = data['ESTU_PRGM_ACADEMICO'].str.contains(pattern)
      data.loc[mask, 'ESTU_PRGM_ACADEMICO'] = 'Finanzas'

  carreras_sociales = [
      'trabajo social', 'ciencia politica', 'sociologia', 'filosofia', 'filologia e idiomas',
      'economia', 'planeacion y desarrollo social', 'ciencia politica y gobierno', 'mercadeo',
      'antropologia', 'relaciones economicas internacionales', 'historia', 'teologia', 'ciencia politica y gobierno',
      'derecho y ciencias politicas', 'gestion empresarial', 'teologia', 'ciencia politica',
      'gestion cultural y comunicativa', 'ciencia politica y relaciones internacionales', 'desarrollo familiar',
      'sociologia', 'politica y relaciones internacionales', 'derecho y ciencias sociales', 'adminsitraci n de empresas',
      'gobierno y asuntos publicos', 'mercadeo internacional y publicidad', 'derecho y ciencias politicas',
      'ciencia politica gobierno y relaciones internacionales', 'psic loga', 'terapias psicosociales', 'antropolog a',
      'pedagog a', 'gesti n cultural', 'psicolog a', 'ciencia pol tica y gobierno', 'relaciones internacionales',
      'derecho y ciencias pol ticas', 'ciencia pol tica y gobierno', 'gerencia de la seguridad y analisis socio politico',
      'gobierno y relaciones internacionales', 'econom a', 'jurisprudencia', 'ciencia pol tica','sociolog a', 'teolog a',
      'derecho', 'desarrollo territorial', 'comercio internacional', 'psicologia', 'periodismo']

  for carrera in carreras_sociales:
      pattern = re.compile(carrera, flags=re.IGNORECASE)
      mask = data['ESTU_PRGM_ACADEMICO'].str.contains(pattern)
      data.loc[mask, 'ESTU_PRGM_ACADEMICO'] = 'Sociales'

  carreras_letras = [
      'traduccion ingles frances espa ol', 'linguistica y literatura', 'linguistica', 'lenguas modernas',
      'letras filologia hispanica', 'espa ol y filologia clasica', 'filosof a', 'literatura', 'filosof a',
      'literatura', 'lenguas y cultura', 'periodismo y opinion publica', 'filosof a y humanidades', 'filologia hispanica',
      'lenguas modernas y cultura']

  for carrera in carreras_letras:
      pattern = re.compile(r'\b' + carrera + r'\b', flags=re.IGNORECASE)
      mask = data['ESTU_PRGM_ACADEMICO'].str.contains(pattern)
      data.loc[mask, 'ESTU_PRGM_ACADEMICO'] = 'Letras'

  carreras_construccion = [
      'construccion', 'construccion y gestion en arquitectura', 'construcciones civiles', 'arquitectura',
      'gestion y desarrollo urbanos', 'construcciones civiles', 'urbanismo', 'conservacion y restauracion de bienes muebles']

  for carrera in carreras_construccion:
      pattern = re.compile(r'\b' + carrera + r'\b', flags=re.IGNORECASE)
      mask = data['ESTU_PRGM_ACADEMICO'].str.contains(pattern)
      data.loc[mask, 'ESTU_PRGM_ACADEMICO'] = 'Construcción'

  carreras_deportes = [
      'deporte', 'entrenamiento deportivo', 'recreacion', 'gesti n deportiva']

  for carrera in carreras_deportes:
      pattern = re.compile(r'\b' + carrera + r'\b', flags=re.IGNORECASE)
      mask = data['ESTU_PRGM_ACADEMICO'].str.contains(pattern)
      data.loc[mask, 'ESTU_PRGM_ACADEMICO'] = 'Deportes'

  carreras_documentacion = [
      'historia y archivistica', 'archivistica', 'sistemas de informacion y documentacion',
      'investigaci n criminal', 'investigacion criminal', 'gesti n de sistemas de informaci n', 'criminalistica']

  for carrera in carreras_documentacion:
      pattern = re.compile(r'\b' + carrera + r'\b', flags=re.IGNORECASE)
      mask = data['ESTU_PRGM_ACADEMICO'].str.contains(pattern)
      data.loc[mask, 'ESTU_PRGM_ACADEMICO'] = 'Documentación'

  carreras_hotelerias = [
      'hoteleria y turismo', 'culinaria y gastronom a', 'turismo', 'hoteleria y turismo ecologico',
      'gastronom a', 'adminsitracion turistica y hotelera']

  for carrera in carreras_hotelerias:
      pattern = re.compile(r'\b' + carrera + r'\b', flags=re.IGNORECASE)
      mask = data['ESTU_PRGM_ACADEMICO'].str.contains(pattern)
      data.loc[mask, 'ESTU_PRGM_ACADEMICO'] = 'Hotelerías'

  carreras_publicidad = [
      'publicidad', 'marketing y logistica', 'publicidad y marketing creativo',
      'logistica empresarial', 'publicidad internacional', 'publicidad y marketing digital', 'marketing']

  for carrera in carreras_publicidad:
      pattern = re.compile(r'\b' + carrera + r'\b', flags=re.IGNORECASE)
      mask = data['ESTU_PRGM_ACADEMICO'].str.contains(pattern)
      data.loc[mask, 'ESTU_PRGM_ACADEMICO'] = 'Publicidad'

  pattern = re.compile(r'^COMUNICACION', flags=re.IGNORECASE)
  mask = data['ESTU_PRGM_ACADEMICO'].str.contains(pattern)
  data.loc[mask, 'ESTU_PRGM_ACADEMICO'] = 'Comunicación'

  pattern = re.compile(r'^ADMINISTRACION', flags=re.IGNORECASE)
  mask = data['ESTU_PRGM_ACADEMICO'].str.contains(pattern)
  data.loc[mask, 'ESTU_PRGM_ACADEMICO'] = 'Administración'

  carrera_ingenieria_software = 'ingeniieria de software'
  pattern = re.compile(r'\b' + carrera_ingenieria_software + r'\b', flags=re.IGNORECASE)
  mask = data['ESTU_PRGM_ACADEMICO'].str.contains(pattern)
  data.loc[mask, 'ESTU_PRGM_ACADEMICO'] = 'Ingeniería'

  encoded_df_programa = pd.get_dummies(data['ESTU_PRGM_ACADEMICO'].astype(str), prefix='PROGRAMA_').astype(int)
  data = pd.concat([data, encoded_df_programa], axis=1)
  data.drop('ESTU_PRGM_ACADEMICO', axis=1, inplace=True)

  return data

def normalize_program(text):
    text = text.lower()
    text = re.sub(r'[^a-z]+', ' ', text)
    return text.strip()

# Resultados:

In [ ]:
df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(normalize_program)

In [ ]:
df = clean_data(df)

In [ ]:
print(df.dtypes)

ID                           int64
FAMI_TIENEINTERNET           int64
ESTU_PAGOMATRICULAPROPIO     int64
FAMI_TIENECOMPUTADOR         int64
RENDIMIENTO_GLOBAL          object
                             ...  
PROGRAMA__Negocios           int64
PROGRAMA__Profesional        int64
PROGRAMA__Publicidad         int64
PROGRAMA__Salud              int64
PROGRAMA__Sociales           int64
Length: 77, dtype: object


In [ ]:
print(df.dtypes['RENDIMIENTO_GLOBAL'])

object


In [ ]:
df

,ID,FAMI_TIENEINTERNET,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,RENDIMIENTO_GLOBAL,Entre 1 millón y menos de 2.5 millones,Entre 2.5 millones y menos de 4 millones,Entre 4 millones y menos de 5.5 millones,Entre 5.5 millones y menos de 7 millones,Entre 500 mil y menos de 1 millón,...,PROGRAMA__Hotelerías,PROGRAMA__Ingeniería,PROGRAMA__Letras,PROGRAMA__Licenciatura,PROGRAMA__Naturales,PROGRAMA__Negocios,PROGRAMA__Profesional,PROGRAMA__Publicidad,PROGRAMA__Salud,PROGRAMA__Sociales
0,904256,1,0,1,medio-alto,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,645256,0,0,1,bajo,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,308367,1,0,0,bajo,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,470353,1,0,1,alto,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,989032,1,0,1,medio-bajo,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,25096,1,1,1,medio-alto,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
692496,754213,1,0,1,bajo,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
692497,504185,1,1,1,medio-bajo,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
692498,986620,0,1,1,bajo,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
mapeo_rangos = {'bajo': 1, 'medio-bajo': 2, 'medio-alto': 3, 'alto': 4}
df['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL'].map(mapeo_rangos)

In [ ]:
# DATOS FALTANTES

print(df.isnull().sum())

ID                          0
FAMI_TIENEINTERNET          0
ESTU_PAGOMATRICULAPROPIO    0
FAMI_TIENECOMPUTADOR        0
RENDIMIENTO_GLOBAL          0
                           ..
PROGRAMA__Negocios          0
PROGRAMA__Profesional       0
PROGRAMA__Publicidad        0
PROGRAMA__Salud             0
PROGRAMA__Sociales          0
Length: 77, dtype: int64


In [ ]:
print(df.columns)

num_columnas = len(df.columns)
print("Número de columnas:", num_columnas)


Index(['ID', 'FAMI_TIENEINTERNET', 'ESTU_PAGOMATRICULAPROPIO',
       'FAMI_TIENECOMPUTADOR', 'RENDIMIENTO_GLOBAL',
       'Entre 1 millón y menos de 2.5 millones',
       'Entre 2.5 millones y menos de 4 millones',
       'Entre 4 millones y menos de 5.5 millones',
       'Entre 5.5 millones y menos de 7 millones',
       'Entre 500 mil y menos de 1 millón', 'Menos de 500 mil',
       'Más de 7 millones', 'No pagó matrícula', 'Horas_Trabajo_0',
       'Horas_Trabajo_Entre 11 y 20 horas',
       'Horas_Trabajo_Entre 21 y 30 horas', 'Horas_Trabajo_Menos de 10 horas',
       'Horas_Trabajo_Más de 30 horas', 'Horas_Trabajo_nan',
       'DEPARTAMENTO__AMAZONAS', 'DEPARTAMENTO__ANTIOQUIA',
       'DEPARTAMENTO__ARAUCA', 'DEPARTAMENTO__ATLANTICO',
       'DEPARTAMENTO__BOGOTÁ', 'DEPARTAMENTO__BOLIVAR', 'DEPARTAMENTO__BOYACA',
       'DEPARTAMENTO__CALDAS', 'DEPARTAMENTO__CAQUETA',
       'DEPARTAMENTO__CASANARE', 'DEPARTAMENTO__CAUCA', 'DEPARTAMENTO__CESAR',
       'DEPARTAMENTO__CHOCO', 'DE

In [ ]:
df

,ID,FAMI_TIENEINTERNET,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,RENDIMIENTO_GLOBAL,Entre 1 millón y menos de 2.5 millones,Entre 2.5 millones y menos de 4 millones,Entre 4 millones y menos de 5.5 millones,Entre 5.5 millones y menos de 7 millones,Entre 500 mil y menos de 1 millón,...,PROGRAMA__Hotelerías,PROGRAMA__Ingeniería,PROGRAMA__Letras,PROGRAMA__Licenciatura,PROGRAMA__Naturales,PROGRAMA__Negocios,PROGRAMA__Profesional,PROGRAMA__Publicidad,PROGRAMA__Salud,PROGRAMA__Sociales
0,904256,1,0,1,3,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,645256,0,0,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,308367,1,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,470353,1,0,1,4,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,989032,1,0,1,2,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,25096,1,1,1,3,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
692496,754213,1,0,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
692497,504185,1,1,1,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
692498,986620,0,1,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


# Predicción

En esta parte se utiliza el modelo de Random Forest para predecir el rendimiento académico global(RENDIMIENTO_GLOBAL) del conjunto de datos.

**Importación de Librerías Necesarias:**

Se importan pandas para manejar los datos, RandomForestClassifier de sklearn.ensemble para el modelo de Random Forest, y cross_val_score de sklearn.model_selection para realizar la validación cruzada.

**Preparación de los Datos**:

* División en Características y Variable Objetivo:se divide el conjunto de datos df en dos partes:

  X: Contiene todas las características predictoras, excluyendo la columna RENDIMIENTO_GLOBAL.

  y: Contiene solo la variable objetivo RENDIMIENTO_GLOBAL.


**Validación Cruzada para Selección del Modelo:**
 * Se define un clasificador de Random Forest sin parámetros específicos, utilizando los valores predeterminados.
 * Se realiza una validación cruzada con 5 particiones (cv=5) para evaluar el rendimiento del modelo. La validación cruzada es una técnica que ayuda a evaluar la robustez del modelo al dividir los datos en varios subconjuntos y entrenar/evaluar el modelo en cada partición.
 * Se imprimen las puntuaciones de cada partición de la validación cruzada y la media de estas puntuaciones, proporcionando una medida de la precisión esperada del modelo.

**Entrenamiento del Modelo y Generación de Predicciones:**

 Finalmente, se entrena el modelo de Random Forest utilizando todas las características (X) y la variable objetivo (y).



In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Dividir los datos en características (X) y variable objetivo (y)
X = df.drop("RENDIMIENTO_GLOBAL", axis=1)  # Características
y = df["RENDIMIENTO_GLOBAL"]  # Variable objetivo


# Paso 2: Validación cruzada para la selección del modelo
rf = RandomForestClassifier()
scores = cross_val_score(rf, X, y, cv=5)  # Validación cruzada con 5 folds
print("Cross-validation scores:", scores)
print("Mean CV score:", scores.mean())

# Paso 3: Entrenamiento del modelo y generación de predicciones
rf.fit(X, y)



Cross-validation scores: [0.33781949 0.33866426 0.337213   0.3367148  0.33923466]
Mean CV score: 0.3379292418772563


RandomForestClassifier()

# Limpiar Test.CSV

En esta parte se prepara el conjunto de datos de prueba (test.csv) para ser utilizado en la predicción con el modelo entrenado.
La función clean_data se aplica al conjunto de datos de prueba para realizar una serie de transformaciones y limpiezas, incluyendo:

* Eliminar columnas innecesarias.
* Convertir valores categóricos a numéricos (e.g., "Si" y "No" a 1 y 0).
* Manejar valores faltantes.
* Realizar codificación one-hot para variables categóricas.
* Agrupar y renombrar los programas académicos en categorías más amplias (e.g., "Ingeniería", "Artes", etc.).

In [ ]:
test_data = pd.read_csv("test.csv")
test_data['ESTU_PRGM_ACADEMICO'] = test_data['ESTU_PRGM_ACADEMICO'].apply(normalize_program)
test_data = clean_data(test_data)

In [ ]:
print(test_data.columns)

Index(['ID', 'FAMI_TIENEINTERNET', 'ESTU_PAGOMATRICULAPROPIO',
       'FAMI_TIENECOMPUTADOR', 'Entre 1 millón y menos de 2.5 millones',
       'Entre 2.5 millones y menos de 4 millones',
       'Entre 4 millones y menos de 5.5 millones',
       'Entre 5.5 millones y menos de 7 millones',
       'Entre 500 mil y menos de 1 millón', 'Menos de 500 mil',
       'Más de 7 millones', 'No pagó matrícula', 'Horas_Trabajo_0',
       'Horas_Trabajo_Entre 11 y 20 horas',
       'Horas_Trabajo_Entre 21 y 30 horas', 'Horas_Trabajo_Menos de 10 horas',
       'Horas_Trabajo_Más de 30 horas', 'Horas_Trabajo_nan',
       'DEPARTAMENTO__AMAZONAS', 'DEPARTAMENTO__ANTIOQUIA',
       'DEPARTAMENTO__ARAUCA', 'DEPARTAMENTO__ATLANTICO',
       'DEPARTAMENTO__BOGOTÁ', 'DEPARTAMENTO__BOLIVAR', 'DEPARTAMENTO__BOYACA',
       'DEPARTAMENTO__CALDAS', 'DEPARTAMENTO__CAQUETA',
       'DEPARTAMENTO__CASANARE', 'DEPARTAMENTO__CAUCA', 'DEPARTAMENTO__CESAR',
       'DEPARTAMENTO__CHOCO', 'DEPARTAMENTO__CORDOBA',
       

# Predecir Test

Esta parte del código realiza la predicción sobre el conjunto de datos de prueba utilizando el modelo previamente entrenado y guarda las predicciones en un archivo CSV para su posterior análisis y evaluación.

**No Seleccionar Columnas Específicas, Usar Todas las Columnas:**

No se realizan selecciones específicas de columnas, se copia todo el conjunto de datos de prueba (test_data) en test_data_selected.

**Realizar Predicciones con el Modelo Entrenado (rf):**

Utilizando el modelo de bosque aleatorio (rf) previamente entrenado, se realizan predicciones sobre el conjunto de datos de prueba (test_data_selected) y se almacenan en la variable predictions.

**Crear un DataFrame para Almacenar las Predicciones:**

Se crea un nuevo DataFrame llamado submission que contiene dos columnas: "ID" (identificador de las muestras en el conjunto de datos de prueba) y "RENDIMIENTO_GLOBAL_PRED" (predicciones realizadas por el modelo).

**Guardar las Predicciones en un Archivo CSV:**

Las predicciones se guardan en un archivo CSV llamado "submission.csv" utilizando el método to_csv() de Pandas. El parámetro index=False evita que se escriban los índices del DataFrame en el archivo CSV.

In [ ]:
print(test_data.columns)

num_columnas_test = len(test_data.columns)
print("Número de columnas:", num_columnas_test)

print(df.columns)

num_columnas_df = len(df.columns)
print("Número de columnas:", num_columnas_df)

Index(['ID', 'FAMI_TIENEINTERNET', 'ESTU_PAGOMATRICULAPROPIO',
       'FAMI_TIENECOMPUTADOR', 'Entre 1 millón y menos de 2.5 millones',
       'Entre 2.5 millones y menos de 4 millones',
       'Entre 4 millones y menos de 5.5 millones',
       'Entre 5.5 millones y menos de 7 millones',
       'Entre 500 mil y menos de 1 millón', 'Menos de 500 mil',
       'Más de 7 millones', 'No pagó matrícula', 'Horas_Trabajo_0',
       'Horas_Trabajo_Entre 11 y 20 horas',
       'Horas_Trabajo_Entre 21 y 30 horas', 'Horas_Trabajo_Menos de 10 horas',
       'Horas_Trabajo_Más de 30 horas', 'Horas_Trabajo_nan',
       'DEPARTAMENTO__AMAZONAS', 'DEPARTAMENTO__ANTIOQUIA',
       'DEPARTAMENTO__ARAUCA', 'DEPARTAMENTO__ATLANTICO',
       'DEPARTAMENTO__BOGOTÁ', 'DEPARTAMENTO__BOLIVAR', 'DEPARTAMENTO__BOYACA',
       'DEPARTAMENTO__CALDAS', 'DEPARTAMENTO__CAQUETA',
       'DEPARTAMENTO__CASANARE', 'DEPARTAMENTO__CAUCA', 'DEPARTAMENTO__CESAR',
       'DEPARTAMENTO__CHOCO', 'DEPARTAMENTO__CORDOBA',
       

In [ ]:
# Paso 2: No seleccionar columnas específicas, usar todas las columnas
test_data_selected = test_data.copy()

predictions = rf.predict(test_data_selected)

submission = pd.DataFrame({'ID': test_data['ID'], 'RENDIMIENTO_GLOBAL_PRED': predictions})
submission.to_csv('submission.csv', index=False)


In [ ]:
submission = pd.read_csv('submission.csv')

print(submission.head())

       ID  RENDIMIENTO_GLOBAL_PRED
0  550236                        4
1   98545                        4
2  499179                        4
3  782980                        3
4  785185                        3


In [ ]:
submission = pd.read_csv('submission.csv')

# Reemplazar los valores numéricos por strings
submission['RENDIMIENTO_GLOBAL_PRED'] = submission['RENDIMIENTO_GLOBAL_PRED'].replace({
    1: 'bajo',
    2: 'medio-bajo',
    3: 'medio-alto',
    4: 'alto'
})

# Renombrar la columna
submission = submission.rename(columns={'RENDIMIENTO_GLOBAL_PRED': 'RENDIMIENTO_GLOBAL'})

# Imprimir las primeras filas para verificar los cambios
print(submission.head())

       ID RENDIMIENTO_GLOBAL
0  550236               alto
1   98545               alto
2  499179               alto
3  782980         medio-alto
4  785185         medio-alto


In [ ]:
submission.to_csv('submission.csv', index=False)